In [1]:
# coding: utf-8

import requests
import csv
from bs4 import BeautifulSoup


def get_the_page(url): #get one page's restaurants info
    
    r = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'})
    html_str = r.text
    mypage = BeautifulSoup(html_str,"html.parser")
    
    myitems = mypage.find_all('li',attrs={'class':'sr1-listing-content-cell pois-restaurant-list-cell '})
    
    titles = []
    likes = []
    locations = []
    prices = []
    countries=[]
    styles=[]
    for myitem in myitems:
        
        titles.append(myitem.find('h2').text.strip())
             
        like=myitem.find('span',attrs={'class':'score score-big highlight'})
        if like is not None:       #some item doesn't have like or other features
            likes.append(like.text)
        else:
            like = '0'
            likes.append(like)
    
        locations.append(myitem.find('div',attrs={'class':'icon-info address'}).find('span').text.strip())
    
        prices.append(myitem.find('div',attrs={'class': 'icon-info icon-info-food-price'}).find('span').text)

    countries_materials=mypage.find_all("ul",attrs={'class':'pois-categoryui-list'})
    country_material=[]
    for material in countries_materials:
        country_material.append(material.find_all('li'))

    for li in country_material:
        countries.append(li[0].text.strip())
        if len(li)>1:
            styles.append(li[1].text.strip())
        else:
            styles.append(li[0].text.strip())

    page_content=[]
    for i in range(0,len(titles)):
        page_content.append([titles[i],locations[i],prices[i],countries[i],styles[i],likes[i]])
 
    return page_content
    

In [2]:
def scrape_pages(url_page1):    #get 17 pages, which is limit of open rice, of the results return from your search
    page1 = get_the_page(url_page1)
    page_next=[]
    for q in range(1,17):
        page_number=str(q+1)
        url_next='{0}&page={1}'.format(url_page1,page_number)
        page_next.extend(get_the_page(url_next))
    
    page_all=page1+page_next
    return page_all

In [9]:
#scrape multiple assembly results from the different search criteria
all_content = []
url_list = ['https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-24&BookingPeriod=Breakfast',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-24&BookingPeriod=Lunch',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-24&BookingPeriod=Tea',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-24&BookingPeriod=Dinner']
for url in url_list:
    all_content.append(scrape_pages(url))

In [15]:
all_content

4

In [28]:
with open('open_rice_temp.csv','w') as f:
    writer = csv.writer(f)
    header = ['Name','Location','Price','Region','Style','Likes']
    writer.writerow(header)
    writer.writerows(all_content[0])
    writer.writerows(all_content[1])
    writer.writerows(all_content[2])
    writer.writerows(all_content[3])

In [ ]:
import pandas
df = pandas.read_csv('open_rice_temp.csv') 

In [30]:
len(df)

932

In [31]:
df = df.drop_duplicates() #remove duplicated rows

In [32]:
len(df)

772

In [33]:
df

,Name,Location,Price,Region,Style,Likes
0,LAB EAT Restaurant & Bar,尖沙咀金巴利道87-89號僑豐大厦地下1-2號舖,$201-400,西式,酒,432
1,Shine,尖沙咀北京道12A號太子集團中心6樓,$201-400,西式,海鮮,687
2,Yadllie Plate,旺角西洋菜街1號兆萬中心11樓,$101-200,西式,酒,484
3,漁獲浜燒 Toretore Hamayaki,銅鑼灣軒尼詩道525號澳門逸園中心18樓,$201-400,韓國菜,韓式炸雞,498
4,The Grill Room,銅鑼灣駱克道459-461號The L. Square 5樓,$201-400,日本菜,日本菜,508
5,Sky726,旺角彌敦道724-726號25樓,$201-400,西式,扒房,367
6,樂園 Happyland,尖沙咀北京道65-69號環球商業大廈2樓205-06室,$101-200,法國菜,甜品/糖水,571
7,Burgeroom,尖沙咀彌敦道132號美麗華廣場一期2 樓 207 號舖,$51-100,西式,樓上cafe,433
8,The Joomak,尖沙咀廣東道3-27號海港城海運大廈3樓LCX 34號舖,$101-200,美國菜,漢堡包,467
9,The Hwaduk,銅鑼灣開平道1號Cubus 21樓,$101-200,韓國菜,沙律,447


In [35]:
df.to_csv('open_rice.csv', encoding='utf-8', index=False)